![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [32]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# Start coding here
# Use as many cells as you need
crimes = pd.read_csv("crimes.csv", dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


In [34]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185715 entries, 0 to 185714
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   DR_NO         185715 non-null  int64 
 1   Date Rptd     185715 non-null  object
 2   DATE OCC      185715 non-null  object
 3   TIME OCC      185715 non-null  object
 4   AREA NAME     185715 non-null  object
 5   Crm Cd Desc   185715 non-null  object
 6   Vict Age      185715 non-null  int64 
 7   Vict Sex      185704 non-null  object
 8   Vict Descent  185705 non-null  object
 9   Weapon Desc   73502 non-null   object
 10  Status Desc   185715 non-null  object
 11  LOCATION      185715 non-null  object
dtypes: int64(2), object(10)
memory usage: 17.0+ MB


In [35]:
crimes_2 = crimes

In [36]:
#Cambiar el nombre de las columnas para un uso mas facil
columns_new_name = {"DATE OCC":"Date Occ",
                    "TIME OCC":"Time Occ",
                    "AREA NAME":"Area Name",
                    "Crm Cd Desc":"Crime Comited",
                    "LOCATION":"Location"}
crimes_2.rename(columns = columns_new_name, inplace=True)
crimes_2.columns

Index(['DR_NO', 'Date Rptd', 'Date Occ', 'Time Occ', 'Area Name',
       'Crime Comited', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Weapon Desc',
       'Status Desc', 'Location'],
      dtype='object')

In [37]:
#Verificar que todos los datos de la columan "Time Occ" tengan 4 digitos
crimes_2["Time Occ"] = crimes_2["Time Occ"].astype(str)
is_nomalize = crimes_2["Time Occ"].str.fullmatch(r'\d{4}').all()
print(is_nomalize)

True


In [38]:
crimes_2["Time Occ"] = crimes_2["Time Occ"].str.slice(0, 2) + ":" + crimes_2["Time Occ"].str.slice(2, 4)
crimes_2["Time Occ"].head()

0    11:10
1    16:20
2    12:00
3    06:35
4    09:00
Name: Time Occ, dtype: object

In [39]:
crimes_2["Time Occ"] = pd.to_datetime(crimes_2["Time Occ"], format='%H:%M')
crime_hour = crimes_2[["DR_NO","Time Occ","Area Name","Vict Age", "Location"]]

In [40]:
crime_hour["Time Occ"] = crime_hour["Time Occ"].dt.strftime('%H:%M:%S')
crime_hour.head()

,DR_NO,Time Occ,Area Name,Vict Age,Location
0,220314085,11:10:00,Southwest,27,2500 S SYCAMORE AV
1,222013040,16:20:00,Olympic,60,3300 SAN MARINO ST
2,220614831,12:00:00,Hollywood,28,1900 TRANSIENT
3,231207725,06:35:00,77th Street,37,6200 4TH AV
4,220213256,09:00:00,Rampart,79,1200 W 7TH ST


In [41]:
#which hour has the highest frequency of crimes
peak_crime_hour = crime_hour["Time Occ"].value_counts().idxmax()
peak_crime_hour = int(peak_crime_hour.split(':')[0])
peak_crime_hour

12

In [42]:
#Which area has the largest frequency of night crime (10 pm and 3:59am)
crime_location_night = crimes_2[["DR_NO","Time Occ","Area Name","Location"]]
crime_location_night["Time Occ"] = crime_location_night["Time Occ"].dt.strftime('%H:%M:%S')

In [43]:
from datetime import time

crime_location_night["Time Occ"] = pd.to_datetime(crime_location_night['Time Occ'], format='%H:%M:%S').dt.time

def is_night(t):
    return (t >= time(22,0)) or (t <= time(3,59))

crime_location_night_2 = crime_location_night[crime_location_night['Time Occ'].apply(is_night)]

In [44]:
peak_night_crime_location = crime_location_night_2["Area Name"].value_counts().idxmax()
peak_night_crime_location

'Central'

In [45]:
#Identify the number of crime comitted againd cictims of diferent age groups
crimes_age = crimes_2[["DR_NO","Vict Age"]]
crimes_age.head()

,DR_NO,Vict Age
0,220314085,27
1,222013040,60
2,220614831,28
3,231207725,37
4,220213256,79


In [46]:
age_bins = [0, 17, 25, 34, 44, 54, 64, np.inf]
age_labels = ["0-17", "18-25", "26-34", "35-44", "45-54", "55-64", "65+"]
crimes["Age Bracket"] = pd.cut(crimes["Vict Age"], bins=age_bins, labels=age_labels)

In [47]:
victim_ages = crimes["Age Bracket"].value_counts()
print(victim_ages)

26-34    47470
35-44    42157
45-54    28353
18-25    28291
55-64    20169
65+      14747
0-17      4528
Name: Age Bracket, dtype: int64
